In [1]:
import sys,os
import torch
import pytorch_lightning as pl

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from models.dVAE.models.model import DiscreteVAE
from models.dVAE.training.lightning import DVAETrainModule
from datasets.weather_bench import WeatherDataset

In [2]:
tensor = torch.zeros(1, 1, 64, 64)

dvae = DiscreteVAE(
    num_tokens=2048,
    codebook_dim=256,
    num_layers=1,
    num_resnet_blocks=2,
    hidden_dim=128,
    channels=1,
    smooth_l1_loss=True,
    temperature=0.2
)

model = DVAETrainModule.load_from_checkpoint('/workspace/Haea_dev/checkpoints/dVAE/2m_temperature/2m_temperature-epoch=32-val_loss=0.00.ckpt')




/workspace/Haea_dev/venv/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'dvae' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['dvae'])`.


In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)
weather = WeatherDataset(0, device=device, offline=True)

lat = (32, 39.3)
lon = (124, 131.4)
input, _, _ = weather.load_2D(variables=['2m_temperature'], latitude=lat, longitude=lon)

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 17545, latitude: 64, longitude: 64,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float32 27.0 27.25 ... 42.5 42.75
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float32 119.2 119.5 ... 135.0
  * time                          (time) datetime64[ns] 2019-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, latitude, longitude) float32 ...
    10m_v_component_of_wind       (time, latitude, longitude) float32 ...
    2m_temperature                (time, latitude, longitude) float32 ...
    geopotential                  (time, level, latitude, longitude) float32 ...
    land_sea_mask                 (latitude, longitude) float32 ...
    mean_sea_level_pressure       (time, latitude, longitude) float32 ...
    ...                

Processing futures: 0it [00:00, ?it/s]

Processing futures: 1it [00:00,  1.93it/s]

0.54582 sec


In [4]:
print(input['2m_temperature'].shape)

dataset = input['2m_temperature']

predict = model(dataset[:2].to(device))

torch.Size([17545, 1, 64, 64])
torch.Size([2, 2048, 32, 32])


In [5]:
print(predict.shape)

torch.Size([2, 1024])
